<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
report

{'0': {'precision': 0.9814126394052045,
  'recall': 0.9777777777777777,
  'f1-score': 0.9795918367346939,
  'support': 270.0},
 '1': {'precision': 0.8064516129032258,
  'recall': 0.8333333333333334,
  'f1-score': 0.819672131147541,
  'support': 30.0},
 'accuracy': 0.9633333333333334,
 'macro avg': {'precision': 0.8939321261542151,
  'recall': 0.9055555555555556,
  'f1-score': 0.8996319839411174,
  'support': 300.0},
 'weighted avg': {'precision': 0.9639165367550067,
  'recall': 0.9633333333333334,
  'f1-score': 0.9635998661759786,
  'support': 300.0}}

In [9]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.9676258992805755,
   'recall': 0.9962962962962963,
   'f1-score': 0.9817518248175182,
   'support': 270.0},
  '1': {'precision': 0.9545454545454546,
   'recall': 0.7,
   'f1-score': 0.8076923076923077,
   'support': 30.0},
  'accuracy': 0.9666666666666667,
  'macro avg': {'precision': 0.961085676913015,
   'recall': 0.8481481481481481,
   'f1-score': 0.8947220662549129,
   'support': 300.0},
  'weighted avg': {'precision': 0.9663

In [10]:
import mlflow 
import mlflow.sklearn
import mlflow.xgboost

In [12]:
# Initialize MLflow

mlflow.set_experiment("Anomaly Detection2(display params)")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/03/24 15:38:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/707497107526165419/runs/2b232a0b784f41bab7af731c45ea9e36
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/707497107526165419


2025/03/24 15:39:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/707497107526165419/runs/f19dd2adce45453196a74da18cb8130c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/707497107526165419


2025/03/24 15:39:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/707497107526165419/runs/ee88797286044a62bc43b46e3cd40c28
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/707497107526165419


2025/03/24 15:39:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/707497107526165419/runs/33ae3f10e3a84d3faa2d79f9b882c0bb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/707497107526165419


### Register the Model

In [13]:
model_name = 'XGB-SMOTE'
run_id=input('Please type RunID')   
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

# we can also register model like this
# result = mlflow.register_model( model_uri , model_name)



# we have to copy 'runid' from MLflow.
# Go to MLflow and select our present experiment("Anomaly Detection2(display params)")
#  from 'Experiments' & click on 'XGClassifier with SMOTE' model(which we've 
#   chosen as best model since it has highest recall-1 compared to other 3 models)
#  & then click on 'overview'.There we find 'Run ID' copy it and paste it 
#  in output box.

# After this our selected model is registered. Which we can see in MLflow by
#  clicking on 'Models' option at top of the page


Successfully registered model 'XGB-SMOTE'.
2025/03/24 15:40:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-SMOTE, version 1


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/707497107526165419/runs/2e3c5f048c324b93ae3a2b17faca0acf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/707497107526165419


Created version '1' of model 'XGB-SMOTE'.


Another way of registering the Model

In [14]:
model_name = 'XGB-SMOTE1'
run_id=input('Please type RunID')   
model_uri = f'runs:/{run_id}/model'

result = mlflow.register_model( model_uri , model_name)



# we have to copy 'runid' from MLflow.
# Go to MLflow and select our present experiment("Anomaly Detection2(display params)")
#  from 'Experiments' & click on 'XGClassifier with SMOTE' model(which we've 
#   chosen as best model since it has highest recall-1 compared to other 3 models)
#  & then click on 'overview'.There we find 'Run ID' copy it and paste it 
#  in output box.

# After this our selected model is registered. Which we can see in MLflow by
#  clicking on 'Models' option at top of the page


Successfully registered model 'XGB-SMOTE1'.
2025/03/24 15:41:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-SMOTE1, version 1
Created version '1' of model 'XGB-SMOTE1'.


### Load the Model & run it for prediction values

In [15]:
model_name = "XGB-SMOTE"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

# when we run tis code , it will go to the MLflow model registry , download 
#  that & then perform the prediction & results the output.

array([0, 0, 0, 0])

### Transition our registered Model to Production stage (then the model will be able to deploy) with production model name "anomaly-detection-prod".
- mlflow.MLflowClient() class allows us to perform operations such as registerng models, managing model versions , and transitioning models to production stages.

In [ ]:
# Here we need not to remember or mention model_version by using '@challenger'

current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-SMOTE' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1742811301083, current_stage='None', description='', last_updated_timestamp=1742811301083, name='anomaly-detection-prod', run_id='2e3c5f048c324b93ae3a2b17faca0acf', run_link='', source='models:/XGB-SMOTE/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

### Load the production Model("anomaly-detection-prod") & run it for prediction values

In [17]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 